According to https://glassboxmedicine.com/2021/02/06/designing-custom-2d-and-3d-cnns-in-pytorch-tutorial-with-code/
What we need is a 2DCNN

I am going to try and carry this team by implementing one.

In [1]:
# importing the libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt

# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score


# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
from torch.utils.data import Dataset
import h5py
# from plot3D import *

#from Demo 2 in the Aformnentioned source code
import timeit     #For keeping track of runtimes 
from src import run_experiment       ## body of code that runs that NN 
import utils   ##in directory



/home/jcristia/eleg404/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class treeConvNN(nn.Module):#Every CNN is defined as a class; inheriting from nn.Module
    def __init__(self): ##Define .conv & .fc  which define the layers that make up the model
        super(treeConvNN,self).__init__() ##gives access to parent class 
        self.conv = nn.Sequential(         ##v 114 wavelength input ===> 100 patterns output
                    nn.Conv2d(in_channels=114,out_channels=228, kernel_size=(3,3), stride=(3,3), padding=0),
                    nn.ReLU(inplace = True),
                    ## output of last in sequence becomes input
                    nn.Conv2d(in_channels=228,out_channels=456, kernel_size=(3,3), stride=(3,3), padding=0),
                    nn.ReLU(inplace = True))
        self.fc = nn.Linear(in_features = 456*3*3,out_features=3)
                            ### not sure about ^this                
    def forward(self,x):   ## How the Model interacts with inputs & outputs
        x = self.conv(x)
        x = torch.flatten(x,start_dim=1,end_dim=-1)
        x = self.fc(x)
        out = {'out':x}
        return out

In [3]:
##reference: https://github.com/rachellea/pytorch-computer-vision/blob/master/load_dataset/custom_tiny.py
class treeData(Dataset):
    def __init__(self,setname):
        self.setname = setname
        assert setname in ['train','val','test']   ## keep this for H5 labels
        #Define dataset                         
        hf = h5py.File("treeVSnot.h5", "r")
        self.all_frames = hf[setname][:] 
        self.all_labels = hf[setname+'Labels'][:]
        hf.close()
        self.label_meanings = ['trees','otherTrees','nonTrees']
    def __len__(self):
        """Return the total number of examples in this split, e.g. if
        self.setname=='train' then return the total number of examples
        in the training set"""
        x = len(self.all_labels) 
        return x  ## edit to be length of data array
        
    def __getitem__(self, idx):    ##keep idx 
        """Return the example at index [idx]. The example is a dict with keys
        'data' (value: Tensor for an RGB image) and 'label' (value: multi-hot
        vector as Torch tensor of gr truth class labels)."""
        selected_frame = self.all_frames[idx]   ##just access the array at idx
        ## this will not work with hyperspec^   Perhaps remove
        #convert image to Tensor and normalize
        image = utils.to_tensor_and_normalize(selected_frame)  ## make tensor for single frame
        ##^THIS THROWS ERROR 
        #load label
        label = torch.Tensor(self.all_labels[idx])  ##just access the array at idx
        
        sample = {'data':image, #preprocessed image, for input into NN
                  'label':label,
                  'img_idx':idx}
        return sample

In [4]:
general_results_dir='./results'   ##results exported here 
    
tot0 = timeit.default_timer()   ##start runtime
run_experiment.DoExperiment(descriptor='trees2dconv',    #for naming files and such
    general_results_dir=general_results_dir,   ##defined above
    custom_net = treeConvNN,       ## My custom model
    custom_net_args = {},
    learning_rate = 1e-3, #default 1e-3
    weight_decay = 1e-7, #default 1e-7
    num_epochs=5, patience = 3,
    batch_size = 1, debug=True,
    use_test_set = False, task = 'train_eval',
    old_params_dir = '',
    chosen_dataset = treeData,     #current data format is [batch,4096pixels,114 wavelengths]
    chosen_dataset_args = {})
tot1 = timeit.default_timer()
print('Total Time', round((tot1 - tot0)/60.0,2),'minutes')

Model-trees2dconv
num_workers = 0
batch_size = 1
device = cuda
Running with optimizer lr=0.001, momentum=0.99 and weight_decay=1e-07
Epoch 0


TypeError: Input tensor should be a float tensor. Got torch.int16.